In [29]:
import xml.etree.ElementTree as etree
import codecs
import csv
import time
import os
import pandas as pd

pd.set_option("display.max_colwidth", 150)

# Parsing xml
parsing xml can be a difficult, I used this video as a referenace
https://www.youtube.com/watch?v=AeRN4zI7Dhk&t=725s

In [30]:
wiki_xml = "./data/leagueoflegends_pages_current.xml"
enc = "utf-8"
parsed_path = "./data/parsed_lol_data.csv"

In [31]:
# get an overview of how xml files are
# and the namespace key for different types of pages

with open(wiki_xml) as f:
    for x in range(70):
        print(f.readline().strip())

<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.11/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.11/ http://www.mediawiki.org/xml/export-0.11.xsd" version="0.11" xml:lang="en">
<siteinfo>
<sitename>League of Legends Wiki</sitename>
<dbname>leagueoflegends</dbname>
<base>http://leagueoflegends.fandom.com/wiki/League_of_Legends_Wiki</base>
<generator>MediaWiki 1.39.5</generator>
<case>first-letter</case>
<namespaces>
<namespace key="-2" case="first-letter">Media</namespace>
<namespace key="-1" case="first-letter">Special</namespace>
<namespace key="0" case="first-letter" />
<namespace key="1" case="first-letter">Talk</namespace>
<namespace key="2" case="first-letter">User</namespace>
<namespace key="3" case="first-letter">User talk</namespace>
<namespace key="4" case="first-letter">League of Legends Wiki</namespace>
<namespace key="5" case="first-letter">League of Legends Wiki talk</namespace>
<namespace key="6" 

In [32]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [33]:
def strip_tag_name(t):
    t = elem.tag
    idx = k = t.rfind("}")
    if idx != -1:
        t = t[idx + 1:]
    return t

In [34]:
totalCount = 0
title = None
start_time = time.time()

In [35]:
title = ''
id = -99
inrevision = False
ns = 0
text = ''
with codecs.open(parsed_path, "w", enc) as parsed_file:
    articlesWriter = csv.writer(parsed_file, quoting=csv.QUOTE_MINIMAL)

    articlesWriter.writerow(['id', 'title', "ns", "text"])
    
    for event, elem in etree.iterparse(wiki_xml, events=('start', 'end')):
        tname = strip_tag_name(elem.tag)
        if event == 'start':
            if tname == 'page': # reset params every page
                title = ''
                id = -99
                inrevision = False
                ns = 0
                text = ''
            elif tname == 'revision':
                # Do not pick up on revision id's
                inrevision = True
            elif tname == 'title':
                title = elem.text
            elif tname == 'id' and not inrevision and elem.text!=None:
                id = int(elem.text)
            elif tname == 'ns' and elem.text!=None:
                ns = int(elem.text)
            elif tname == 'text' and elem.text!=None:
                text = elem.text.replace("\n", "").strip()
        elif (event == "end") & (tname == 'text') & (text == "") & (elem.text!=None): # some text only appear in end
            text = elem.text.replace("\n", " ").strip()
        elif tname == 'page':
            totalCount += 1
            articlesWriter.writerow([id, title, ns, text])
            if totalCount > 1 and (totalCount % 100000) == 0:
                print("{:,}".format(totalCount))
        elem.clear()
        
time_took = time.time() - start_time
print(f"Total runtime: {hms_string(time_took)}")

100,000
200,000
300,000
400,000
500,000
600,000
700,000
800,000
900,000
1,000,000
1,100,000
1,200,000
1,300,000
Total runtime: 0:00:58.88


# Cleaning

In [53]:
df = pd.read_csv(parsed_path)

In [54]:
df

,id,title,ns,text
0,1297,User:Angela,2,{{wikia:User:Angela}}[[cs:Uživatel:Angela]][[de:Benutzer:Angela]][[fr:Utilisateur:Angela]][[pl:Użytkownik:Angela]]
1,1300,League of Legends Wiki:Administrators,4,#REDIRECT [[League_of_Legends_Wiki:User_Rights]]
2,1452,League of Legends Wiki:Copyrights,4,":'''''Please see [[w:Wikia:Licensing|Wikia:Licensing]] for details of the licensing on this wiki.''''':'''<span style=""color:red; font-size: 150%;..."
3,1460,Category:Browse,14,"This is the top-level or root category.Ideally, every other category should be a subcategory of at least one other, and every article should be in..."
4,1462,Forum:Index,110,Welcome to the {{SITENAME}} forum! This forum can be used by members of this community to discuss topics related to their wiki.:''Notes: You may h...
...,...,...,...,...
1300474,1632022,File:Smolder MMOOOMMMM! HD.png,6,{{Image header/ability}}
1300475,1632023,"File:Smolder Flap, Flap, Flap HD.png",6,{{Image header/ability}}
1300476,1632024,File:Noxus TFT bronze icon.png,6,== Licensing =={{Fairuse}}[[Category:Trait images]]
1300477,1632025,File:Noxus TFT gold icon.png,6,== Licensing =={{Fairuse}}[[Category:Trait images]]


In [55]:
df[df["title"].str.contains(".*Yasuo.*", na=False)]

,id,title,ns,text
803876,1059922,File:Yasuo OriginalSquare.png,6,{{Image header/skin}}
804062,1060156,Yasuo/LoL,0,{{Section top}} {{Game banner|Yasuo}} {{LoL navigation}} {{Infobox champion|Yasuo}} {{LoL navigation mobile}} {{Infobox stats|Yasuo}} {{clr}} == A...
804063,1060157,Talk:Yasuo/@comment-9365255-20131115132101,1,OOOOoooooooow my GWWWWWWWWWWaaaaaaaaad!!!11111I bet he will be super cool and overpowered and stuff.Like totally!100% for real!
804064,1060158,Talk:Yasuo/@comment-12617334-20131115160503,1,hello
804095,1060197,Talk:Yasuo/@comment-17834311-20131115193054,1,hi.
...,...,...,...,...
1287362,1618735,File:Yasuo TrueDamageTFTSquare.png,6,{{Image header/skin}}
1287363,1618736,File:Yasuo TrueDamageSquareTFT.png,6,NaN
1287580,1618954,File:Hwei Original FirstEncounterYasuo.ogg,6,== Licensing =={{Fairuse}}[[Category:Hwei voice-overs]]
1288396,1619782,File:Yasuo PROJECTTFTSquare.png,6,{{Image header/skin}}


From looking into the namespaces keys, we can remove lot's of pages based on their namespace,
those pages don't have lore data, most of them are blogs, forums, wiki metadata etc

In [56]:
df["ns"].value_counts()

ns
1       659913
6       204364
3       124768
501      84557
1201     72913
1200     57180
2        31330
0        19992
828      12833
500       9411
14        6842
2001      6465
10        5283
111       3644
110        339
4          240
8          185
5          112
15          56
1202        18
11           8
502          7
829          6
2000         5
7            3
2900         3
710          2
Name: count, dtype: int64

In [57]:
df[df["ns"] == 710]

,id,title,ns,text
1151792,1474824,Yuumi 083.ogg.en.srt,710,~Where's my fishy fish fish dish?~ *meow!*
1161125,1485301,Yasuo Original R 0.ogg.en.srt,710,sorye ge ton!


In [58]:
# remove based on name space
df = df[~df["ns"].isin([-2,-1,1,2,3,4,5,6,7,8,9,10,11,12,13,15,110,111,114,115,420,421,500,501,502,503,828,829,1200,1201,1202,2000,2001,2002,2300,2301,2302,2900,2901])]

In [59]:
# remove those rows
display(df[df["text"].str.contains("#REDIRECT.* |#redirect.*", na=False)])
df = df[~df["text"].str.contains("#REDIRECT.* |#redirect.*", na=False)] 

,id,title,ns,text
123,2020,Main Page,0,#REDIRECT [[League of Legends Wiki]]
143,2054,Summoners,0,#REDIRECT [[Summoner]]
149,2067,Finished item,0,#REDIRECT [[Legendary item]]
166,2085,Creeps,0,#REDIRECT [[Minion (League of Legends)]]
168,2104,Creep,0,#REDIRECT [[Minion (League of Legends)]]
...,...,...,...,...
1299202,1630750,Cosmic Pearl (Consumable) (The Path of Champions),0,#REDIRECT [[Cosmic Pearl (The Path of Champions)]]
1299204,1630752,Greater Cosmic Pearl (Consumable) (The Path of Champions),0,#REDIRECT [[Greater Cosmic Pearl (The Path of Champions)]]
1299223,1630771,Portal Pal (The Path of Champions),0,#REDIRECT [[Portal Pals (The Path of Champions)]]
1299573,1631121,Epic Quest (The Path of Champions),0,#REDIRECT [[Spirit_Forge_(The_Path_of_Champions)]]


In [60]:
# remove anything with title talk
display(df[df["title"].str.contains("Talk:.*|talk:.*", na=False)])
df = df[~df["title"].str.contains("Talk:.*|talk:.*", na=False)]

,id,title,ns,text
3836,7628,User talk:80.217.140.153,0,"{| style=""margin-top:1em; width=""100%"" height:600px; border:1px solid #2C1807; font-size:95%; text-align:center; padding:0; border:1px solid black..."
13498,20831,User talk:201.226.192.11,0,"{| style=""margin-top:1em; width=""100%"" height:600px; border:1px solid #2C1807; font-size:95%; text-align:center; padding:0; border:1px solid black..."
15547,23969,Talk:Twitch/@comment-189.100.235.81-20100912122757/@comment-81.182.48.158-20101002124626,0,"And the olimpic skin was for sale for a limited time, at the winter olimpic"
19968,30716,Talk:Poppy/@comment-89.204.137.102-20101028084437/@comment-24.193.148.25-20101029043023,0,"Its an autoattack +60% of her AP. The high AP ratio is one of the reasons AP Poppy is awesome. Plus, you deal alot of damage using charge with AP ..."
21776,33357,Talk:Katarina/@comment-62.97.212.167-20101106132417/@comment-139.130.57.34-20101113013736,0,Yeah...
...,...,...,...,...
1078110,1391529,Talk:Karma/@comment-24971427-20190401022007/@comment-24971427-20190401055100,0,"I often get Ionian boots on her.You are right though, her early game is quite weak. I just try to clear the wave instead."
1080311,1394282,Talk:Rammus/@comment-37514365-20190323105855/@comment-37331249-20190512042635,0,ok
1082226,1396272,Talk:List of champions/@comment-26612064-20190612145049/@comment-3974211-20190612150720,0,It's not adding 1. It is incorrect because Qiyana has already been added to the Released Champions category. I imagine that was also the issue bef...
1129239,1450369,Talk:Lillia/@comment-4820583-20200715231525/@comment-46223578-20200719014654,0,"Wow, such a boring opinion. After reading it I'm not interested in anything you have to say"


In [61]:
# remove first few rows as they are related to wiki not lol
display(df.iloc[:20])
df = df.iloc[20:]

,id,title,ns,text
3,1460,Category:Browse,14,"This is the top-level or root category.Ideally, every other category should be a subcategory of at least one other, and every article should be in..."
5,1463,Category:Help,14,"'''For a full list of help pages, see [[Help:Contents]],''' which includes non-local help pages, automatically transcluded from [[w:c:help|Wikia H..."
6,1470,Category:Forums,14,The category is for pages relating to the [[Forum:Index|Forum]].[[Category:Organization]]
12,1509,Category:Candidates for deletion,14,{{Administration category|hidden=yes}}Articles in this category have been labelled for deletion. If you wish to discuss the deletion of these page...
14,1517,Category:Policy,14,"This is a list of policy pages for this Wikia. Please also see [[:Category:Help]], and the policy pages on the [[Wikia:Category:Policy|Central Wi..."
19,1672,Category:Copyright,14,Category can include subcategories and articles that have copyright implications. See also [[:Category:Policy]].[[Category:Organization]]
28,1753,Category:Community,14,"This category may include pages and subcategories related to the active contributors to the site, as seen on [[Special:ListUsers]].[[Category:Orga..."
29,1766,Category:Templates,14,This is the base category for '''templates'''. Templates should be placed in appropriate subcategories.See [[Project:Templates]] for collated info...
33,1776,Category:Article stubs,14,{{Administration category|hidden=yes}}This category lists '''stub articles''' on this wiki.A stub is a short article that can be expanded with mor...
41,1802,Category:Category templates,14,Templates used on category pages.{{Templatecategory}}[[Category:Templates]]


In [62]:
df

,id,title,ns,text
130,2030,League of Legends,0,"{{Portal/LOL}} {{Cleanup| * Game modes section links to [[Classic]], which is legacy terminology. * Game modes section includes TFT but describes ..."
131,2032,Champion,0,"<!--{{Stub|This page needs more on the characteristics champions have, or links to the relevant pages for such things.}}--> :''This is about the {..."
133,2040,Urf,0,"[[File:Urf OriginalSkin.jpg|thumb|300px|Urf, the Manatee]] : ''Not to be confused with {{tip|Ultra Rapid Fire}}'' : ''For other terms, visit [[Urf..."
135,2043,Nunu/LoL,0,{{Section top}}{{Game banner|Nunu|disp_name=Nunu & Willump}}{{LoL navigation}}{{Infobox champion|Nunu|disp_name=Nunu & Willump}}{{LoL navigation m...
136,2044,Alistar/LoL,0,{{Section top}}{{Game banner|Alistar}}{{LoL navigation}}{{Infobox champion|Alistar}}{{LoL navigation mobile}}{{Infobox stats|Alistar}}{{clr}}== Ab...
...,...,...,...,...
1299648,1631196,Succubus's Brand (The Path of Champions),0,"<!--The temporary, non-official code for this relic is RX001-->{{PoC item infobox|custom|name=Succubus's Brand|image=Succubus's Brand LoR relic.pn..."
1299649,1631197,Voidborne Carapace (The Path of Champions),0,"<!--The temporary, non-official code for this relic is RX002-->{{PoC item infobox|custom|name=Voidborne Carapace|image=Voidborne Carapace LoR reli..."
1299650,1631198,Arcane Knowledge (The Path of Champions),0,"<!--The temporary, non-official code for this item is IX005-->{{PoC item infobox|custom|name=Arcane Knowledge|image=06RU043-full.png|imagesize=200..."
1299651,1631199,Immortal (The Path of Champions),0,"<!--The temporary, non-official code for this item is IX006-->{{PoC item infobox|custom|name=Immortal|image=08NX028-full.png|imagesize=200px|type=..."


In [63]:
df.to_csv("data/parsed_cleaned_data.csv", index=False)